### 自然语言处理之 文本分析

In [1]:
#  Copyright private in 2018 
#  Modify Date: 
#       2018 - 9 - 19
#  Purpose : 
#       Text Analysise  by fasttext/word2vec/Deep learning/LSTM
# ----------
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import pandas as pd 
import sklearn 
import sys 
print ("TensorFlow need run under:\n",sys.version)

TensorFlow need run under:
 3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [ ]:
# hierarchical softmax - 类别较多时，通过构建哈夫曼编码来技术softmax layer 计算 和之前的word2vec 的trick
# N-gram - 之使用unigram 的话会丢掉word order 信息

In [3]:
# 当文本量大时，可以通过fasttext来学习
# data Exampe: 
#__label__2,.......content ......
#__label__3,......content......
#__label__4,......content......

# Data Category 
# 1. car 2.sports 3.entertainment 4. technology 5. military

## (一). 通过Facebook 工业界fasttext 模型根据输入的新闻内容预测该新闻所属的种类

## 有监督学习 - 新闻分类/或者用于用户情感的褒贬分析

### 1. 生成文本数据

In [6]:
import jieba
import random 
cate_dic = {'technology':1, 'car':2, 'entertainment':3, 'military':4, 'sports':5}

# data set dict 
# df_technology
df_technology  = pd.read_csv("./data/technology_news.csv", encoding = 'utf-8')
df_technology  = df_technology.dropna()  # 空的字符drop 掉

# df_car
df_car  = pd.read_csv("./data/car_news.csv", encoding = 'utf-8')
df_car  = df_car.dropna()

# df_entertainment 
df_entertainment   = pd.read_csv("./data/entertainment_news.csv", encoding = 'utf-8')
df_entertainment   = df_entertainment.dropna()

# df_sprots
df_sports   = pd.read_csv("./data/sports_news.csv", encoding = 'utf-8')
df_sports   = df_sports.dropna()

#df_military 
df_military  = pd.read_csv("./data/military_news.csv",encoding = 'utf-8')
df_military  = df_military.dropna()


# 提取出一定量的数据
# .values -> array 数组
# .tolist -> list 列表
# [1000:21000] -> 切片找出一部分的数据
technology = df_technology.content.values.tolist()[1000:21000]
car        = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[1000:21000]
military   = df_military.content.values.tolist()[1000:21000]
sports     = df_sports.content.values.tolist()[1000:21000]


In [8]:
print (technology[12])

　　现在家里都拉了网线，都能无线上网，一定要帮他们先登上WiFi，另外，老人不懂得流量是什么，也不知道如何开关，控制流量，所以设置好流量上限很重要，免得不小心点开了视频或者下载，电话费就大发了。


In [10]:
print car[3]

　　伴随全新途观L的上市，上汽大众大众品牌刷新了SUV产品谱系，构建起包含途观丝绸之路版、全新途观L及Teramont途昂在内的SUV产品矩阵，覆盖多个SUV细分市场，为消费者提供更为丰富的购车选择与更优质的汽车生活，助力上汽大众赢得更亮眼的市场表现。


In [11]:
print entertainment[20]

　　基于优酷强势内容，霸屏客厅的阿里家庭娱乐春节期间发出花式大招，精心编排，巧妙聚合，“春晚热播，明星扎堆，心疼爸妈，今年绝不熬着看”。花式看春晚第一弹是“明星组团儿上春晚”，囊括沈腾、贾玲、曹云金、小沈阳、冯巩、郭德纲、宋小宝、岳云鹏、青岛大姨、赵本山、蔡明、潘长江、郭冬临、黄宏、巩汉林共15位春晚笑匠，集合每个笑星历年春晚或综艺作品，将其精彩演出片段集结成辑打包放出。第二弹是以专题的形式聚合2017年春晚语言/歌曲/魔术戏曲三大类型，横向打穿，爱看小品的老爸老妈，追星的迷妹迷弟一键直达。


In [12]:
print military[90]

　　报道称，美国一些防务专家批评白宫错失了一些适当采取军事手段但不至于引发战争的机会。


In [13]:
print sports[82]

　　,中新网,清远1月20日电 (记者 唐贵江)2017/18“叮咚出行”广东超级杯七人制足球联赛(清远杯)暨2017青联杯足球联赛启动仪式，20日在清远清新体育馆足球场举行，中国足球协会副主席、广东省足球协会名誉主席、广东省民间足球促进会会长容志行为赛事授牌，并勉励活跃群众足球运动。


### 2. Remove Stopwards

In [9]:
import pandas as pd
stopwords_read  = pd.read_csv("./data/stopwords_NLP.txt", index_col = False, quoting = 3, sep = "\t", names = ['stopword'], encoding = 'utf-8')
stopwords  = stopwords_read['stopword'].values


In [10]:
#help(pd.read_csv)

In [13]:
print (stopwords[10:112])

['+' ',' '-' '--' '.' '..' '...' '......' '...................' './' '.一'
 '记者' '数' '年' '月' '日' '时' '分' '秒' '/' '//' '0' '1' '2' '3' '4' '5' '6' '7'
 '8' '9' ':' '://' '::' ';' '<' '=' '>' '>>' '?' '@' 'A' 'Lex' '[' '\\' ']'
 '【' '】' '^' '_' '`' 'exp' 'sub' 'sup' '|' '}' '~' '~~~~' '·' '×' '×××' 'Δ'
 'Ψ' 'γ' 'μ' 'φ' 'φ．' 'В' '—' '——' '———' '‘' '’' '’‘' '“' '”' '”，' '…' '……'
 '…………………………………………………③' '′∈' '′｜' '℃' 'Ⅲ' '↑' '→' '∈［' '∪φ∈' '≈' '①' '②'
 '②ｃ' '③' '③］' '④' '⑤' '⑥' '⑦' '⑧' '⑨' '⑩' '──']


### 3. Data Text Preparation 

In [15]:
def preprocess_text(content_lines, sentences, category,stopwords):
    sentences = []
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x)>1,segs)  # len(x)<=1 为空，过滤掉
            segs = filter(lambda x: x not in stopwords, segs)  # x in stopwards 过滤掉
            # category 按照字典cate_dic 里面的value 取1/2/3/4 and so on 
            sentences.append("__label__"+str(category)+", "+" " .join(segs))
        except Exception as e:
            print ("Exception infor",e)
            #print line
            continue
        

## 生成训练数据
sentences = []

# preprocess the data 
preprocess_text(technology,sentences,cate_dic['technology'],stopwords)
preprocess_text(car,sentences,cate_dic['car'],stopwords)
preprocess_text(entertainment,sentences,cate_dic['entertainment'],stopwords)
preprocess_text(military,sentences,cate_dic['military'],stopwords)
preprocess_text(sports,sentences,cate_dic['sports'],stopwords)

# 乱序处理 - 使得同一类别的样本不扎堆出现在一起
random.shuffle(sentences)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Yazhou\AppData\Local\Temp\jieba.cache
Loading model cost 1.622 seconds.
Prefix dict has been built succesfully.


### 4.写入文本操作

In [16]:
print "writing data to fasttext format"
openout  = open('training_datasets.txt','w')  # 写入的方式打开

for sentence in sentences:
    openout.write(sentence.encode('utf-8') + "\n")  # 中文形式  encode('utf-8') + "\n" 换行符
print "Done...."

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-16-f322c0dbf4dc>, line 1)

### 5.调用fasttext 训练生成模型

In [22]:
import fasttext
classifier  = fasttext.supervised('training_datasets.txt','classifier.model', label_prefix  = '__label__') 

ImportError: No module named fasttext

In [ ]:
# test and evaluted the data set 
testresult = classifier.test('training_datasets.txt')

# printing 准确值和召回率
print 'testresult precision', testresult.precision 
print 'testresult recall', testresult.recall 

print 'Number of examples: ', testresult.nexamples


### 6.实际预测过程

In [ ]:
label_to_cata_test = {'technology':1, 'car':2, 'entertainment':3, 'military':4, 'sports':5}

# 待测试样本
texts  = ['马来西亚 反贪 委员会 称 马来西亚 前总理 纳吉布 被逮捕 因其牵涉']
labels = classifier.predict(texts)

#printing label and category 
print('labels is :', labels)
print label_to_cata_test[int(labels[0][0])]


# 同时输出有多少的概率来肯定种类是这个
labels = classifier.predict_proba(texts)
print labels 

In [ ]:
### TopK 预测结果分析
# K = 5
category = classifier.predict(texts, K = 5)
print category

In [ ]:
# 输出category 及其对应的概率
category = classifier.predict_proba(texts, K = 5)
print category

## (二). 通过fasttext做无监督文本学习

In [6]:
# Data Preparation 
def preprocess_text_unsupervised(content_lines, sentences, category):
    sentences = []
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x)>1,segs)  # len(x)<=1 为空，过滤掉
            segs = filter(lambda x: x not in stopwards, segs)  # x in stopwards 过滤掉
            # category 按照字典cate_dic 里面的value 取1/2/3/4 and so on 
            sentences.append(" ".join(segs))
    except Exception,e:
        print line
        continue
        

## 生成训练数据
sentences = []

# preprocess the data 
preprocess_text(technology,sentences,cate_dic['technology'])
preprocess_text(car,sentences,cate_dic['car'])
preprocess_text(entertainment,sentences,cate_dic['entertainment'])
preprocess_text(military,sentences,cate_dic['military'])
preprocess_text(sports,sentences,cate_dic['sports'])


# print out 
print "writing data to  fasttext unsupervised learning format ..."
writeout = open('unsupervised_trainingdatasets.txt','w')

for sentence in sentences: 
    writeout.write(sentence.encode('utf-8')+ "\n")
    
print"write Done ..."



In [ ]:
# Using fasttext to training the data sets
# Skipgram model
model = fasttext.skipgram('unsupervised_trainingdatasets.txt','model')
# print the list 
print model.words

# CBOW model - continue bags of words 
model  = fasttext.cbow('unsupervised_trainingdatasets.txt','model')
print model.words # list of words from dictionary 

## (三). Gensim vs Fasttext

In [ ]:
# Data Preparation 
def preprocess_text_unsupervised(content_lines, sentences, category):
    sentences = []
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x)>1,segs)  # len(x)<=1 为空，过滤掉
            segs = filter(lambda x: x not in stopwards, segs)  # x in stopwards 过滤掉
            # category 按照字典cate_dic 里面的value 取1/2/3/4 and so on 
            sentences.append(" ".join(segs))
    except Exception,e:
        print line
        continue
        

## 生成无监督训练数据
sentences = []

# preprocess the data - 无监督不需要标签
preprocess_text(technology,sentences)
preprocess_text(car,sentences)
preprocess_text(entertainment,sentences)
preprocess_text(military,sentences)
preprocess_text(sports,sentences)

In [ ]:
##model fitting 
model = Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)
model.save(gensim_word2vec.model)
model.wv['信息']

## (四).文本分类by Deep Learning  

In [ ]:
# Note: 
#并不是将全部数据全部加在到内容，而是将一个batch 一个batch 学习及权重更新去学习新的模型
# 

### 1.CNN 做文本分类

In [7]:
# used on short text processing 
# LSTM can be used to long text processing

# CNN 中的filter 窗口大小跟词向量的文本大小有关，每个词的窗口可能为 词向量的个数* 每个词向量的维数
# 窗口filter + pooling 池化 + fullconnection 全链接


### 1.1 Data preprocessing 

In [18]:
# Data processing and remove stopwards 
import pandas as pd 

# data set dict 
# df_technology
df_technology  = pd.read_csv("./data/technology_news.csv", encoding = 'utf-8')
df_technology  = df_technology.dropna()  # 空的字符drop 掉

# df_car
df_car  = pd.read_csv("./data/car_news.csv", encoding = 'utf-8')
df_car  = df_car.dropna()

# df_entertainment 
df_entertainment   = pd.read_csv("./data/entertainment_news.csv", encoding = 'utf-8')
df_entertainment   = df_entertainment.dropna()

# df_sports
df_sports   = pd.read_csv("./data/sports_news.csv", encoding = 'utf-8')
df_sports   = df_sports.dropna()

#df_military 
df_military  = pd.read_csv("./data/military_news.csv",encoding = 'utf-8')
df_military  = df_military.dropna()


# 提取出一定量的数据
# .values -> array 数组
# .tolist -> list 列表
# [1000:21000] -> 切片找出一部分的数据
technology = df_technology.content.values.tolist()[1000:21000]
car        = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[1000:21000]
military   = df_military.content.values.tolist()[1000:21000]
sports     = df_sports.content.values.tolist()[1000:21000]


### remove stopwards
stopwords = pd.read_csv("data/stopwords_NLP.txt",index_col = False, quoting =3, sep= "\t", names =['stopwords'],encoding = 'utf-8' )
stopwords = stopwords['stopwords'].values

In [19]:
### Construct Data 
# Data Preparation 
import  jieba 
def preprocess_text_cnn(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x)>1,segs)  # len(x)<=1 为空，过滤掉
            segs = filter(lambda x: x not in stopwords, segs)  # x in stopwards 过滤掉
            # category 按照字典cate_dic 里面的value 取1/2/3/4 and so on 
            sentences.append((" ".join(segs),category))  # 添加数据及其label
        except Exception as e:
            print (">>>",line)
            print ("Exception infor>>>",e)
            continue

In [ ]:
import time 
sentences = []
start = time.time()
preprocess_text_cnn(technology,sentences,'technology')
preprocess_text_cnn(car,sentences,'car')
preprocess_text_cnn(entertainment,sentences,'entertainment')
preprocess_text_cnn(military,sentences,'military')
preprocess_text_cnn(sports,sentences,'sports')
end  = time.time()
print ("运行时间: %f(s)" %(end - start) )

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Yazhou\AppData\Local\Temp\jieba.cache
Loading model cost 2.655 seconds.
Prefix dict has been built succesfully.


In [66]:
#sentences

In [80]:
"分割数据集"
from  sklearn.cross_validation import train_test_split

# 拉链 将词语和label 分别分给x and y 
x,y = zip(*sentences)

# split the data into trianing and test data sets 
X_train,X_test,y_train,y_test   = train_test_split(x,y,random_state = 200)

In [81]:
y_test

['technology',
 'sports',
 'entertainment',
 'technology',
 'car',
 'technology',
 'military',
 'technology',
 'sports',
 'technology',
 'technology',
 'technology',
 'sports',
 'sports',
 'sports',
 'entertainment',
 'technology',
 'entertainment',
 'military',
 'technology',
 'military',
 'entertainment',
 'car',
 'car',
 'entertainment',
 'sports',
 'military',
 'military',
 'car',
 'car',
 'car',
 'military',
 'military',
 'car',
 'car',
 'entertainment',
 'entertainment',
 'sports',
 'entertainment',
 'sports',
 'technology',
 'car',
 'sports',
 'car',
 'sports',
 'car',
 'car',
 'military',
 'technology',
 'sports',
 'technology',
 'military',
 'military',
 'sports',
 'sports',
 'sports',
 'entertainment',
 'technology',
 'technology',
 'sports',
 'sports',
 'technology',
 'military',
 'car',
 'sports',
 'technology',
 'sports',
 'military',
 'military',
 'entertainment',
 'entertainment',
 'sports',
 'sports',
 'entertainment',
 'technology',
 'sports',
 'entertainment',
 'sport

In [82]:
y_train

['car',
 'military',
 'entertainment',
 'technology',
 'sports',
 'entertainment',
 'sports',
 'sports',
 'entertainment',
 'entertainment',
 'technology',
 'sports',
 'military',
 'military',
 'car',
 'entertainment',
 'entertainment',
 'technology',
 'technology',
 'car',
 'technology',
 'sports',
 'entertainment',
 'military',
 'sports',
 'military',
 'sports',
 'sports',
 'sports',
 'car',
 'entertainment',
 'military',
 'military',
 'sports',
 'entertainment',
 'entertainment',
 'technology',
 'military',
 'technology',
 'sports',
 'sports',
 'technology',
 'entertainment',
 'entertainment',
 'entertainment',
 'sports',
 'military',
 'sports',
 'car',
 'entertainment',
 'car',
 'car',
 'sports',
 'technology',
 'car',
 'military',
 'car',
 'military',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'sports',
 'military',
 'car',
 'sports',
 'technology',
 'entertainment',
 'military',
 'entertainment',
 'sports',
 'sport

### 1.2 构建神经网络过程 - 中文文本分类 on CPU

In [6]:
# for python2 need to import lib
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function 

import argparse
import sys
import pandas as pd 
import numpy as np
from sklearn import metrics 
import tensorflow as tf 
learn  = tf.contrib.learn 


In [7]:
# Variables Initialization

FLAGS = None
# 文档最长的长度
MAX_DOCUMENT_LENGTH = 100

# 最小词频数
MIN_WORD_FREQUENCY = 2 

# 词嵌入的维度
EMBEDDING_SIZE= 20

# filter 数量
N_FILTERS = 10

# Windows size 
WINDOWS_SIZE = 20

#filter 的形状
FILTER_SHAPE1 = [WINDOWS_SIZE, EMBEDDING_SIZE]
FILTER_SHAPE2 = [WINDOWS_SIZE, N_FILTERS]

# Pooling 
POOLING_WINDOW  = 4
POOLING_STRIDE  = 2 
n_words = 0

In [8]:
# define CNN Model 卷积神经网络
def cnn_model(features,target):
    ###
    ### 两层的卷积神经网络，用于短文本分类
    # 先把词转成词嵌入
    # 我们得到一个形状为[n_words,EMBEDDING_SIZE] 的词表映射矩阵
    # 接着我们可以把一批文本映射成[batch_size,sequence_length,EMBEDDING_SIZE]的矩阵
    
    # one - hot 编码 
    target = tf.one_hot(target,15,1,0)
    
    # 将feature/文本 的序列做一个映射，编成一个二维向量
    word_vectors = tf.contrib.layers.embed_sequence(features, vocab_size = n_words, embed_dim = EMBEDDING_SIZE,scope = 'words')
    
    # 将2维转成3 维
    word_vectors = tf.expand_dims(word_vectors,3)
    
    with tf.variable_scope('CNN_Layer1'):
        # 添加一个二维的卷积滤波
        conv1  = tf.contrib.layers.convolution2d(word_vectors,N_FILTERS,FILTER_SHAPE1,padding = 'VALID')
        # 添加RELU非线性 - 激活函数
        conv1  = tf.nn.relu(conv1)
        
        # maxmimum pooling 
        pool1 = tf.nn.max_pool(conv1,ksize = [1,POOLING_WINDOW,1,1], strides = [1,POOLING_STRIDE,1,1], padding = 'SAME')
        
        # 对矩阵转置 以满足形状
        pool1 = tf.transpose(pool1,[0,1,3,2])
        
    
    with tf.variabel_scope('CNN_Layer2'):
            # 第2个卷积层
            conv2 = tf.contrib.layers.convolution2d(pool1,N_FILTERS,FILTER_SHAPE2,padding = 'VALID')
            
            # 抽取特征
            pool2  = tf.squeeze(tf.reduce_max(conv2,1), squeeze_dims = [1])
            
            
    # FullConnection - 全链接
    # 预测值 : logits
    logits = tf.contrib.layer.fully_connected(pool2, 15, activation_fn = None)  # 无激活函数
    loss   = tf.losses.softmax_cross_entropy(target, logits)  #target:真实值， logits；预测值
    
    
    # 循环迭代
    train_op = tf.contrib.layers.optimize_loss(loss,tf.contrib.framwork.get_gloabl_step(), optimizer = 'Adam', learning_rate = 0.01)
    
    # return
    return({
        'class': tf.argmax(logits,1),
        'prob': tf.nn.softmax(logits)
    }, loss, train_op)

        
        

### 1.3 Tensorflow.preprocessing 里的VocabularyProcessor

In [58]:
temp  = ['I am good', 'You are  here ','I am glad', 'it is great']
#
# 只要出现的最小频率是1 或者比1 大，就处理
vocab_processor  = learn.preprocessing.VocabularyProcessor(10, min_frequency=1)
list(vocab_processor.fit_transform(temp))

# I am good -> [1,2,0,0,......] length = 10

[array([1, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)]

In [62]:
global n_words
# 处理词汇
vocab_processor  = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH,min_frequency = 1)
x_train = np.array(list(vocab_processor.fit_transform(X_train)))
x_test = np.array(list(vocab_processor.fit_transform(X_test)))

n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' %n_words)


Total words: 67311


In [91]:
cate_dic = {'technology':1,'car':2,'entertainment':3, 'military':4, 'sports':5}
cate_dic['entertainment']

3

In [95]:
#-------------
# 将类别映射成数字
import pandas 
cate_dic = {'technology':1,'car':2,'entertainment':3, 'military':4, 'sports':5}
y_train = map(lambda x:cate_dic[x],y_train)
y_test = map(lambda x:cate_dic[x],y_test)

#y_train2 =  pandas.Series(y_train1)
#y_test  =  pandas.Series(y_test)

In [106]:
xy_map = map(lambda x,y:x*y,[1,2,3],[4,5,6])
for i in xy_map:
    print ("xy_map>>> ",xy_map)### Python3中，这个是迭代器，不会打印出结果

"下面是在python2中运行的结果："
"""
('output>>> ', [4, 10, 18])
('output>>> ', [4, 10, 18])
('output>>> ', [4, 10, 18])
"""

xy_map>>>  <map object at 0x000000001FEFA358>
xy_map>>>  <map object at 0x000000001FEFA358>
xy_map>>>  <map object at 0x000000001FEFA358>


"\n('output>>> ', [4, 10, 18])\n('output>>> ', [4, 10, 18])\n('output>>> ', [4, 10, 18])\n"

In [105]:
type(y_train1)

map

In [5]:
# 构建模型
classifier = learn.SKCompat(sklearn.Estimator(model_fn = cnn_model))

# 训练和预测
classifier.fit(x_train,y_train,steps = 1000)
y_predicted = classifier.predict(x_test)['class']

score = metrics.accuracy_score(y_test,y_predicted)
print('Accuracy:{0:f}'.format(score))


NameError: name 'learn' is not defined

### 1.4 LSTM/GRU - RNN 循环神经网络

In [13]:
# 使用RNN完成文本分类
# for python2 need to import lib
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function 

import argparse
import sys
import pandas as pd 
import numpy as np
from sklearn import metrics 
import tensorflow as tf 
from tensorflow.contrib.layers.python.layers import encoders 
learn  = tf.contrib.learn
FLAGS = None


In [15]:
# 通过词袋模型来一批一批的把数据灌进去
MAX_DOCUMENT_LENGTN = 15
MIN_WORD_FREQUENCE  = 1
EMBEDDING_SIZE      = 50

global n_words
# 处理词汇
#vocab_processor  = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH,min_frequency = 1)
vocab_processor  = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH,min_frequency=MIN_WORD_FREQUENCE)
x_train = np.array(list(vocab_processor.fit_transform(X_train)))
x_test = np.array(list(vocab_processor.fit_transform(X_test)))

n_words = len(vocab_processor.vocabulary_)
print('Total words: %d'%n_words)

def bag_of_words_model(feature,target):
    # 生成词袋模型
    target = tf.one_hot(target,15,1,0)
    features = encoders.bow_encoder(features,vocab_size = n_words,embed_dim = EMBEDDING_SIZE)

    # FullConnection - 全链接
    # 预测值 : logits
    logits = tf.contrib.layer.fully_connected(pool2, 15, activation_fn = None)  # 无激活函数
    loss   = tf.losses.softmax_cross_entropy(target, logits)  #target:真实值， logits；预测值
    
    
    # 循环迭代
    train_op = tf.contrib.layers.optimize_loss(loss,tf.contrib.framwork.get_gloabl_step(), optimizer = 'Adam', learning_rate = 0.01)
  
    # return
    return({
        'class': tf.argmax(logits,1),
        'prob': tf.nn.softmax(logits)
    }, loss, train_op)

#
model_fn = bag_of_words_model
classifier = learn.SKCompat(learn.Estimator(model_fn = model_fn))

# 训练和预测
classifier.fit(x_train,y_train,steps = 1000)
y_predicted = classifier.predict(x_test)['class']

score = metrics.accuracy_score(y_test,y_predicted)
print('Accuracy:{0:f}'.format(score))

NameError: name 'X_train' is not defined